# S3 Trigger Setup - Automatic Document Processing with ADE

In [1]:
# ---
# LandingAI Applied AI Content Notebook Template
# ---
# Title: S3 Trigger Setup - Automatic Document Processing with ADE
# Author: Ava Xia
# Description: Streamlined notebook for testing and using the deployed Lambda function
# Target Audience: [Developers, Partners, Customers]
# Content Type: [Tutorial, How-To]
# Publish Date: 2025-09-23
# ADE Version: v0.1.5
# Change Log:
#    - v1.0: Initial draft
#    - v1.1: Modularized with utility functions
# ---

This notebook configures automatic processing when documents are uploaded to S3.

**⚠️ IMPORTANT:** Make sure your AWS session is in the same region as your Lambda function and S3 bucket (in our case `us-east-2`). 

If using AWS SSO, login with:
```bash
aws configure sso
aws sso login --profile your-profile-name
```

## How S3 Triggers Work

```
┌──────────┐      ┌──────────┐      ┌──────────┐      ┌──────────┐
│  Upload  │ ---> │    S3    │ ---> │  Trigger │ ---> │  Lambda  │
│   PDF    │      │  Bucket  │      │  Event   │      │ Process  │
└──────────┘      └──────────┘      └──────────┘      └──────────┘
                                           │                 │
                                           ↓                 ↓
                                    Automatic         Save Results
```

When you upload a PDF to the configured S3 folder:
1. S3 generates an event
2. Event triggers Lambda function
3. Lambda processes the document
4. Results saved to `ade-results/` folder

In [2]:
# Import consolidated utilities and configuration
import os
import json
from datetime import datetime
from pathlib import Path
import pandas as pd
from IPython.display import display, JSON

# Import consolidated modules
from config import get_settings
from utils import (
    setup_aws_environment,
    list_s3_files,
    check_lambda_environment,
    setup_s3_trigger,
    get_lambda_invocation_stats,
    get_error_logs
)

# Initialize environment using config.py and .env
print("="*60)
print("🔧 Initializing AWS environment...")
print("="*60)

# Load configuration (automatically reads from .env)
config, clients, AWS_ACCOUNT_ID, aws_session = setup_aws_environment()

# Check if credentials are valid
if AWS_ACCOUNT_ID in ['EXPIRED', 'ERROR']:
    print("\n⚠️  Please refresh your AWS credentials:")
    print("   aws sso login --profile your-profile-name")
    print("   export AWS_DEFAULT_REGION=us-east-2")
else:
    # Extract configuration values
    BUCKET_NAME = config['bucket_name']
    FUNCTION_NAME = config['function_name']
    ECR_REPO = config['ecr_repo']
    AWS_REGION = config['aws_region']
    
    print("\n✅ Environment ready!")
    print(f"   Lambda: {FUNCTION_NAME}")
    print(f"   Bucket: {BUCKET_NAME}")
    print(f"   Region: {AWS_REGION}")
    print("="*60)

🔧 Initializing AWS environment...
✅ AWS Environment configured
   Profile: workload-dev-2
   Region: us-east-2
   Account: 9700XXXX1993

✅ Environment ready!
   Lambda: ade-lambda-s3
   Bucket: cf-mle-testing
   Region: us-east-2


## 2. Configure S3 Trigger

Set up automatic processing for a specific folder.

In [3]:
# Configure S3 trigger for invoices folder
trigger_folder = "invoices/"

print(f"🎯 Setting up S3 trigger for: {trigger_folder}")
print()

success = setup_s3_trigger(
    clients['s3'],
    clients['lambda'],
    BUCKET_NAME,
    FUNCTION_NAME,
    folder=trigger_folder
)

if success:
    print("\n✅ S3 trigger is active!")
    print(f"   Any PDF uploaded to s3://{BUCKET_NAME}/{trigger_folder}")
    print("   will be automatically processed")

🎯 Setting up S3 trigger for: invoices/

ℹ️  S3 permission already exists
✅ S3 trigger configured!
   📤 Upload PDFs to: s3://cf-mle-testing/invoices/
   ⚡ They will auto-process with Lambda

✅ S3 trigger is active!
   Any PDF uploaded to s3://cf-mle-testing/invoices/
   will be automatically processed


## 3. Test the Trigger

Upload a test file to verify the trigger works.

In [4]:
# Test file upload
import time  # Add missing import

test_file_path = "input_folder/invoice_1.pdf"  # Local file
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
s3_key = f"{trigger_folder}test_trigger_{timestamp}.pdf"

print("📤 Uploading test file...")
print(f"   From: {test_file_path}")
print(f"   To: s3://{BUCKET_NAME}/{s3_key}")
print()

try:
    # Upload the file
    clients['s3'].upload_file(test_file_path, BUCKET_NAME, s3_key)
    print("✅ File uploaded successfully!")
    print("⏳ Waiting for Lambda to process...")
    
    # Wait for processing
    time.sleep(10)
    
    # Check for results
    results = list_s3_files(
        clients['s3'],
        BUCKET_NAME,
        "ade-results/",
        max_files=5
    )
    
    if results:
        print("\n✅ Processing complete!")
        print("   Recent results:")
        for r in results[:3]:
            print(f"   • {r['File']}")
    else:
        print("⚠️ Still processing or no results yet")
        print("   Check CloudWatch logs for details")
        
except FileNotFoundError:
    print(f"❌ Test file not found: {test_file_path}")
    print("   Please update the path to a valid PDF file")
except Exception as e:
    print(f"❌ Upload failed: {e}")

📤 Uploading test file...
   From: input_folder/invoice_1.pdf
   To: s3://cf-mle-testing/invoices/test_trigger_20250925_153109.pdf

✅ File uploaded successfully!
⏳ Waiting for Lambda to process...
📂 Files in s3://cf-mle-testing/ade-results/
Found 5 files

✅ Processing complete!
   Recent results:
   • ade-results/batch_extracted_20250924_195832.json
   • ade-results/batch_extracted_20250925_013951.json
   • ade-results/batch_extracted_20250925_014051.json


## 4. Monitor Lambda Invocations

Check Lambda statistics to see trigger activity.

In [5]:
# Get Lambda invocation stats
stats = get_lambda_invocation_stats(
    clients['logs'],
    FUNCTION_NAME,
    hours_back=1  # Last hour
)

if stats.get('total_invocations', 0) > 0:
    print(f"\n📊 Recent Activity (last hour):")
    print(f"   Invocations: {stats['total_invocations']}")
    print(f"   Success rate: {stats.get('success_rate', 0):.0f}%")
else:
    print("\nℹ️ No recent invocations")
    print("   Upload a file to trigger processing")

📊 Lambda Invocation Statistics (last 1 hours)
   Total Invocations: 1

📊 Recent Activity (last hour):
   Invocations: 1
   Success rate: 100%


## 5. Check for Errors

In [6]:
# Check for recent errors
errors = get_error_logs(
    clients['logs'],
    FUNCTION_NAME,
    hours_back=1
)

if not errors:
    print("✅ No errors in the last hour")
else:
    print(f"⚠️ Found {len(errors)} errors")
    print("   Check CloudWatch logs for details")

❌ Error Logs (last 1 hours)
✅ No errors found
✅ No errors in the last hour


## 6. Remove S3 Trigger

If needed, remove the S3 trigger configuration.

In [7]:
# Remove S3 trigger (uncomment to run)
"""
print("🗑️ Removing S3 trigger...")

try:
    # Clear bucket notifications
    clients['s3'].put_bucket_notification_configuration(
        Bucket=BUCKET_NAME,
        NotificationConfiguration={}
    )
    print("✅ S3 trigger removed")
except Exception as e:
    print(f"❌ Error removing trigger: {e}")
"""

print("ℹ️ To remove triggers, uncomment the code above")

ℹ️ To remove triggers, uncomment the code above


## 7. Best Practices

### Folder Organization
```
s3://bucket/
├── invoices/        # Auto-process with invoice schema
├── receipts/        # Auto-process with receipt schema
├── documents/       # Auto-process with parsing mode
├── ade-results/     # Processing results
└── archive/         # Processed files (optional)
```

### Tips
1. **Use specific folders** for different document types
2. **Monitor CloudWatch** for processing logs
3. **Set up SNS** for processing notifications
4. **Archive processed files** to avoid reprocessing
5. **Use versioning** on S3 bucket for safety